# **Objetivo do Projeto**

Este projeto tem como objetivo demonstrar, de forma prática e acessível, como os embeddings semânticos podem ser utilizados para identificar produtos similares com base em suas descrições. Utilizando um modelo de linguagem pré-treinado e a biblioteca FAISS, mostramos como transformar textos em vetores e realizar buscas por similaridade de forma inteligente, indo além da correspondência literal de palavras.

# **O que é FAISS?**
FAISS é uma ferramenta criada pelo Facebook que ajuda a encontrar itens parecidos de forma rápida e eficiente. No nosso projeto, usamos o FAISS para comparar os vetores dos produtos e descobrir quais são os mais semelhantes entre si. Ele funciona como um "GPS de similaridade", ajudando a localizar os produtos mais próximos em significado dentro de um grande conjunto de dados.

# **O que é o modelo MPNet?**
MPNet é um modelo de linguagem treinado para entender o significado de frases e textos. Ele transforma uma descrição de produto em um vetor de números que representa o que aquele texto quer dizer. Diferente de uma busca por palavras exatas, o MPNet consegue perceber que "camiseta esportiva" e "blusa para academia" têm sentidos parecidos, mesmo com palavras diferentes. Isso permite fazer recomendações mais inteligentes e próximas do que a pessoa realmente procura.

# **Instalação e importação de bibliotecas**

Instala a biblioteca FAISS, usada para buscas rápidas de similaridade.

In [ ]:
%pip install -q faiss-cpu

Importa bibliotecas essenciais: manipulação de dados, uso de modelo de linguagem e construção de índice FAISS.

In [ ]:
import pandas as pd
import torch
import numpy as np
import faiss
import joblib
from sentence_transformers import SentenceTransformer

# **Carrega os dados dos produtos**
Lê a base com as descrições dos produtos.

In [ ]:
url = "https://raw.githubusercontent.com/ksddavila/epe2025_treinamento_ia/main/base_produtos_utensilios_domesticos.csv"
df_produtos = pd.read_csv(url)


In [ ]:
df_produtos.head()

# **Escolhe e carrega o modelo de linguagem**
O modelo **all-mpnet-base-v2** converte textos em vetores numéricos (chamados embeddings) que capturam o significado do texto.

In [ ]:
encoder_model_name = "all-mpnet-base-v2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_model = SentenceTransformer(encoder_model_name, device=device)

# **Função que transforma texto em embedding**
Essa função recebe um texto e retorna um vetor numérico que representa seu significado. Isso será usado para comparar produtos de forma semântica, não apenas por palavras exatas.

In [ ]:
def get_text_embedding(text: str) -> np.ndarray:
    if not text or pd.isna(text):
        return None
    embedding = embedding_model.encode(
        text,
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    return embedding.astype(np.float32)

## **Aplica a função aos produtos**
Para cada descrição de produto, geramos um embedding. Removemos os que falharam.

In [ ]:
df_produtos['embedding'] = df_produtos['TEXTO_DESCRITIVO'].apply(get_text_embedding)
df_local = df_produtos[df_produtos["embedding"].notnull()].copy()

In [ ]:
df_produtos.head()

# **Prepara os dados para o FAISS**
Agora temos dois itens: uma lista de códigos e uma matriz de vetores representando os significados de cada descrição.

In [ ]:
product_codes = df_local["CODIGO_PRODUTO"].tolist()
product_embeddings = np.stack(df_local["embedding"].values).astype("float32")

# Normaliza os vetores (importante para usar o FAISS corretamente)
product_embeddings = product_embeddings / np.linalg.norm(product_embeddings, axis=1, keepdims=True)

# **Cria o índice de busca FAISS**
Esse índice permite encontrar rapidamente os produtos mais similares com base nos seus significados (e não por palavras exatas!).

In [ ]:
embedding_dim = product_embeddings.shape[1]
faiss_index_products = faiss.IndexFlatIP(embedding_dim)
faiss_index_products.add(product_embeddings)

# **Cria um dicionário auxiliar**
Usamos esse dicionário para recuperar rapidamente o vetor de um produto específico.

In [ ]:
product_emb_dict = {
    row["CODIGO_PRODUTO"]: np.array(row["embedding"], dtype=np.float32)
    for _, row in df_local.iterrows()
}

# **Salva o índice e os dados auxiliares**
Agora tudo está salvo em disco. Podemos usar depois sem recalcular.

In [ ]:
faiss.write_index(faiss_index_products, "faiss_produtos.index")
joblib.dump(product_codes, "faiss_product_codes.joblib")
joblib.dump(product_emb_dict, "product_emb_dict.joblib")

print(f"Índice FAISS salvo com {len(product_codes)} produtos.")

# **(Opcional) Recarrega os dados salvos**

In [ ]:
faiss_index_products = faiss.read_index("faiss_produtos.index")
product_codes = joblib.load("faiss_product_codes.joblib")
product_emb_dict = joblib.load("product_emb_dict.joblib")

# **🔎 Função de busca por similaridade**
Essa função permite buscar os produtos mais semelhantes semanticamente a outro, com base em suas descrições.

In [ ]:
def buscar_similares_produto(codigo_produto_consulta,
                              faiss_index,
                              product_codes,
                              product_emb_dict,
                              df_produtos,
                              k=3):
    if codigo_produto_consulta not in product_emb_dict:
        print(f"Aviso: Código {codigo_produto_consulta} não encontrado.")
        return []

    descricao_base = df_produtos.loc[df_produtos["CODIGO_PRODUTO"] == codigo_produto_consulta, "DESCRICAO_MATERIAL"].values[0]
    print(f"\n🔍 Produto consultado: {codigo_produto_consulta} - {descricao_base}")

    query_vector = product_emb_dict[codigo_produto_consulta].reshape(1, -1)
    D, I = faiss_index.search(query_vector, k + 1)

    similares = []
    print("\n📦 Produtos similares:")
    for idx, score in zip(I[0], D[0]):
        codigo_similar = product_codes[idx]
        if codigo_similar == codigo_produto_consulta:
            continue
        descricao_similar = df_produtos.loc[df_produtos["CODIGO_PRODUTO"] == codigo_similar, "DESCRICAO_MATERIAL"].values[0]
        print(f"→ {codigo_similar} | Similaridade: {score:.4f} | {descricao_similar}")
        similares.append((codigo_similar, descricao_similar, float(score)))

        if len(similares) >= k:
            break

    return similares

# **🚀 Exemplo de uso**
Aqui buscamos os 3 produtos mais similares ao produto de código 170759.

In [ ]:
produto_pesquisa = 170759

In [ ]:
resultados = buscar_similares_produto(
    codigo_produto_consulta=produto_pesquisa,
    faiss_index=faiss_index_products,
    product_codes=product_codes,
    product_emb_dict=product_emb_dict,
    df_produtos=df_produtos
)